In [12]:
from video.models import ShowSource, MovieSource, Show, Movie
from asgiref.sync import sync_to_async
import os
import django
import requests
import json
from pyquery import PyQuery as pq
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [13]:
HEADERS = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50"
                         ".0.2661.102 Safari/537.36"}


In [14]:
sources = ShowSource.objects.filter(server='sendvid')

In [34]:
failed_sources = []

In [35]:
for index, source in enumerate(sources):
    if not source.hls_source:
        url = source.url_source if 'http' in source.url_source else 'http:' + source.url_source
        print(index, 'source: ', source.id, url)
        response = requests.get(url, headers=HEADERS)
        # print(pq(response.content).find('video source').attrib['src'])
        if len(pq(response.content).find('video source')) > 0:
            src = pq(response.content).find('video source')[0].attrib['src']
            print(src)
            source.hls_source = src
            source.save()
        else:
            failed_sources.append(source.id)
            print('----- FAIL!!! source: ', source.id)
            print('----- source not working', source.chapter.name)

181 source:  6521 http://sendvid.com/embed/2mfziyqb
----- FAIL!!! source:  6521
----- source not working King Crab: Space Crustacean
1715 source:  8141 http://sendvid.com/embed/phgmzoh5
----- FAIL!!! source:  8141
----- source not working Rescatate soldado braian
2935 source:  9397 http://sendvid.com/embed/41ul6e9r
----- FAIL!!! source:  9397
----- source not working Conductor designado / Ardillas al amanecer
2936 source:  9398 http://sendvid.com/embed/fdfhmfp5
----- FAIL!!! source:  9398
----- source not working El árbol de Navidad / Noche de graduación / Punchline
3338 source:  9804 http://sendvid.com/embed/w9dn2a1m

3339 source:  9805 http://sendvid.com/embed/lqiq9ft6

3342 source:  9808 http://sendvid.com/embed/bm9ln1vl

4029 source:  10526 http://sendvid.com/embed/w37tkjiz
----- FAIL!!! source:  10526
----- source not working Episodio 12
4374 source:  10873 http://sendvid.com/embed/dyjdq3yh



In [21]:
source = sources[182]
url = source.url_source if 'http' in source.url_source else 'http:' + source.url_source
response = requests.get(url, headers=HEADERS)


[<source#video_source>]

In [22]:
len(pq(response.content).find('video source'))

1

In [36]:
sourcesFails = ShowSource.objects.filter(id__in=failed_sources)
sourcesFails.count()

5

In [38]:
for fail in sourcesFails:
    print(fail.id, fail.chapter.show.name, fail.chapter.season, fail.chapter.number_chapter , fail.chapter.name)
    print(fail.url_source)

6521 what a cartoon 2 3 King Crab: Space Crustacean
//sendvid.com/embed/2mfziyqb
8141 Alejo y Valentina 1 8 Rescatate soldado braian
//sendvid.com/embed/phgmzoh5
9397 Animaniacs 5 7 Conductor designado / Ardillas al amanecer
//sendvid.com/embed/41ul6e9r
9398 Animaniacs 5 8 El árbol de Navidad / Noche de graduación / Punchline
//sendvid.com/embed/fdfhmfp5
10526 (Des)encanto 2 2 Episodio 12
//sendvid.com/embed/w37tkjiz
